In [1]:
SOURCE = "../bbb3_glass_eval_111716.log"

In [2]:
""" Install instructions
Always un in jupyterlab

- sudo apt install nodejs # didn't work
- curl -o- https://raw.githubusercontent.com/nvm-sh/nvm/v0.38.0/install.sh | bash
- nvm install --lts
- sudo chmod a+rwx /usr/local/share/jupyter/lab/
- mkdir /usr/local/share/jupyter/lab/extensions
- jupyter labextension install @jupyter-widgets/jupyterlab-manager --debug # will need to chmod some stouff to make work

- jupyter nbextension enable --py widgetsnbextension
"""

In [26]:
import numpy as np
import cv2
1440*1200
blob=np.zeros((800,12000,1440),np.uint16).reshape(36000)
cv2.imwrite("testblob.png", blob)
del blob
blob=cv2.imread("testblob.png")

Known bugs:
 - Code is inconsistent on whether scan period is the whole period or just for the active part of the scan (hack corrected)


In 112016 the gyro(or odom rotation? not sure) readings need to be multiplied by  0.78(+-0.02) to be accurate(not tested on other runs)

The intensity seems to fall off at a rate of 218.3 (unknown uncert) units per meter for the Hokuyo. Perhaps it reports in a log scale? Not normalized in any case.
Below two meters, the intensity is pretty weird. 2-10 m is where the intensity seems to be sane. Somewhere between 0.5 and 1.0 m seems to be where most things report peak reflectivity

Localization seems to be accurate to +-10 mrad.

In [3]:
import numpy as np
%matplotlib widget
import matplotlib.pyplot as plt
import os

In [4]:
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_unfixed.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_earlyts.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_moving_hiqual.rays")
# rayfile = os.path.join(SOURCE, "raw_lidar_dumps","rays","rays_manyparticle_highrecall.rays")

rayfile = "/home/anuran/repos/Vulcan/bin/rayDump.rays" # HACK

In [5]:
rays_struct = np.dtype(
    [
        ('sx', np.float32),
        ('sy', np.float32),
        ('ex', np.float32),
        ('ey', np.float32),
        ('r' , np.float32),
        ('th', np.float32),
        ('i',  np.uint16),
        ('pad',  np.uint16),
        ('idx',  np.int32),
        ('ts', np.float64), #remove for old ray file formats
      ])

# rays = np.memmap(rayfile,rays_struct, 'r',1)
rays0 = np.fromfile(rayfile,rays_struct)
# Clip rays0 to a multiple of 1081 rays, under the assumption that a scan is 1081 rays long
if len(rays0)%1081 != 0:
    print("WARNING: file is not made of complete scans!")
    print("Remainder:", len(rays0)%1081)
# For the new sensors, we will need to find a different way to split scans
rays0 = rays0[:int(len(rays0)//1081*1081)]

Remainder: 952


In [7]:
# import matplotlib.pyplot as plt
# from mpl_toolkits.mplot3d import Axes3D

# skip = 100   # Skip every n points
%gui qt
# fig = plt.figure()
# ax = fig.add_subplot(111, projection='3d')
import mayavi.mlab
mayavi.mlab.close(all=True)
# fig = mayavi.mlab.figure(bgcolor=(1.0, 1.0, 1.0), size=(640, 360))
# mayavi.mlab.points3d(rays0['ex'], rays0['ey'], rays0['th'],rays0['th'], colormap='spectral',mode= 'point')
# mayavi.mlab.test_plot3d()
# mayavi.mlab.close(all=True)

In [17]:
rays1=rays0[rays0['idx']==0]
rays2=rays0[rays0['idx']==1]
rays0['ts'][-1]-rays0['ts'][0]

438.99257000013404

In [9]:
%matplotlib widget
# plt.imshow(np.diff(rays0['ts'].reshape(-1,1081)[:1000]-rays0['ts'][0],axis=0))
# d=(d+np.pi/2)%
# plt.plot(np.diff(rays0['th'].reshape(-1,1081)[:1000],axis=0)[:,100],'b,')
# plt.plot(rays1['ts'],rays1['th'],'b,')
nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
woof=np.diff(rays1['th'].reshape(-1,1081))
dts=np.diff(rays1['ts'].reshape(-1,1081))
dth=np.diff(nan_th.reshape(-1,1081))

slope=dth/dts
slope=slope[np.abs(dth)<np.pi/2]
stopped=[]
rangle=[]
for row,rts in zip(nan_th.reshape(-1,1081),rays1['ts'].reshape(-1,1081)):
    use = np.unwrap(row[np.isfinite(row)])
    use_ts = rts[np.isfinite(row)]
    if len(use)==0:
        print('err')
        print(row)
        break
    slope = (use[-1]-use[0])/(use_ts[-1]-use_ts[0])
    off=np.mean(use-(use_ts-rts[0])*slope)
    stopped.append(row-(rts-rts[0])*slope-off)
    rangle.append((rts-rts[0])*slope+off)
stopped = np.vstack(stopped)
rangle = np.vstack(rangle)
flat_rangle = np.unwrap(rangle[:,0].ravel()*2)/2
# plt.plot(rays1['ts'][::1081],flat_rangle,'b,')
_=plt.hist(np.diff(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081])),1000)
plt.gca().set_yscale("log")
# plt.plot((stopped.ravel()+np.pi)%(2*np.pi)-np.pi,'b,')
# expect_th = (rays1['ts']-rays1['ts'][0])*np.float64(np.median(slope[slope>0]))%(2*np.pi)
# plt.plot(rays1['ts'][rays1['r']>1],(expect_th-rays1['th'])[rays1['r']>1]%(2*np.pi),'b,')
# plt.plot(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081]))
# plt.hist(slope[np.isfinite(np.abs(slope))],1000)

<ipython-input-9-e50cb4296749>:6: RuntimeWarning: divide by zero encountered in divide
  nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
<ipython-input-9-e50cb4296749>:6: RuntimeWarning: invalid value encountered in divide
  nan_th=rays1['th']/(rays1['r']>1.0)/(rays1['r']<30.0)
/home/anuran/.local/lib/python3.8/site-packages/numpy/lib/function_base.py:1447: RuntimeWarning: invalid value encountered in subtract
  a = op(a[slice1], a[slice2])
<ipython-input-9-e50cb4296749>:30: RuntimeWarning: invalid value encountered in divide
  _=plt.hist(np.diff(np.diff(flat_rangle)/np.diff(rays1['ts'][::1081])),1000)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:
%matplotlib widget
# TODO: transition to floating point timestamps, not microseconds
odofile = "/home/anuran/repos/Vulcan/bin/odoDump.rays" # HACK
odo_struct = np.dtype(
    [
        ('ts', np.float64),#TODO: this is a double now!
        ('x', np.float32),
        ('y', np.float32),
        ('th', np.float32),
        ('pad' , np.float32),
        ('lx', np.float32),
        ('ly', np.float32),
      ])
def odosub(odo):
    fields = ['ts','x','y','th']
    for f in fields:
        odo[f]-=odo[0][f]
    

# rays = np.memmap(rayfile,rays_struct, 'r',1)
poses = np.fromfile(odofile,odo_struct)
imu= poses[0::3]
odo= poses[1::3]
lp = poses[2::3]

In [11]:
# This cell checks odometry for smoothness
%matplotlib widget
def d_dt(t,x,dt,order=1):
    if order>1:
        x=d_dt(t,x,dt,order-1)
    if order==0:
        return x
    dt = dt/2
    lot = np.clip(t-0.001*dt, t[0], t[-1])
    hit = np.clip(t+dt, t[0], t[-1])
    assert(np.all(hit-lot > 0))
    lox = np.interp(lot,t,x)
    hix = np.interp(hit,t,x)
    return (hix-lox)/(hit-lot)
# first, resample to 100hz
odo_struct_rt = np.dtype(
    [
        ('ts', np.float64),
        ('x', np.float32),
        ('y', np.float32),
        ('th', np.float32),
        ('pad' , np.float32),
        ('lx', np.float32),
        ('ly', np.float32),
      ])
ots = (odo['ts']-odo['ts'][0])/1e6
hz=2
assert(np.all(np.diff(ots)>=0))
tsr = np.max(ots)-np.min(ots)
ts100 = np.linspace(ots[0],ots[-1],np.maximum(int(tsr*hz),2))
odo100 = np.zeros_like(ts100,dtype=odo_struct_rt)
odo100['ts'] = ts100
for key in ['x','y','th','lx','ly']:
    odo100[key]=np.interp(ts100,ots,odo[key])

for hz in [40,20,10,5,2]:
    # now check smoothness
#     plt.plot(np.unwrap(odo['th']))
    dk_dt = d_dt(ots,np.unwrap(odo['y']),1/hz,2)#/(d_dt(ots,np.unwrap(odo['th']),1/(2*hz))+0.1)
    plt.plot(ots,(dk_dt),)
    print(np.max(dk_dt))



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

756087864.8878378
2554686165.6213484
2653254129.686415
2700661017.166998
-346858747.92639446


In [10]:
%matplotlib widget

# odosub(odo)
# plt.plot(lp['ts']/1e6,np.sqrt(lp['x']**2+lp['y']**2), ',')
# plt.plot(lp['ts']/1e6,lp['th'], ',')

# plt.plot(rays0['ts'],np.sqrt(rays0['sx']**2+rays0['sy']**2), ',')
# plt.plot(rays0['ts'][647::1081],rays0['th'][647::1081], ',')
# plt.plot(np.arange(1081*20)%1081,rays0['r'][1081*200:1081*200+1081*20],',')

plt.plot(odo['ts']/1e6,odo['x'],'b')


# plt.plot(odo['x'],odo['y'], ',')
# ax1 = plt.subplot(211)
# ax2 = plt.subplot(212,sharex=ax1)
# ax1.plot(odo['ts'][:-1]/1e6,np.diff(np.unwrap(odo['th'])), ',')
# ax2.plot(rays0['ts'][500::1081][:-1],(np.diff(rays0['th'][500::1081])+np.pi/4)%(np.pi/2)-np.pi/4, ',')

# ax2.plot(odo['ts']/1e6,odo['x'], ',')

# plt.plot(odo['ts']/1e6,np.unwrap(odo['th']),',')
# plt.plot(imu['ts']/1e6,np.unwrap((imu['th'])))
plt.plot(odo['ts']/1e6,np.unwrap(odo['th']),'b')
plt.plot(odo['ts']/1e6,np.unwrap(lp['th']),'r')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
# As a hack to split scans up by sensor, we will look for sudden large jumps in the ray 
# starting position. As long as the localization doesn't jump too much, this hack should work.
# Plot the cdf of jumps to find a good cutoff.
# (as of this writing a 0.5m jump looks like a good cutoff)
%matplotlib widget

tmp=np.sqrt(np.diff(rays0['sx'],append=[0])**2+np.diff(rays0['sy'],append=[0])**2)
plt.plot(np.sort(tmp),',')

SENSOR_JUMP_THRESHOLD=0.5 # by inspection of the plot

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:
# Use index to read sensor (uncomment block after if you want to use position hack instead
rays1=rays0[rays0['idx']==0]
rays2=rays0[rays0['idx']==1]

# # Use the jump threshold to detect changes of sensor
# restarts=np.sqrt(np.diff(rays0['sx'],append=[0])**2+np.diff(rays0['sy'],append=[0])**2)>SENSOR_JUMP_THRESHOLD

# # scans=rays0[:len(rays0)-len(rays0)%2162].reshape(-1,2162)
# raysr = rays0[restarts]# HACK to separate scans, gives the ray right before a restart
# idx=np.arange(len(restarts))[restarts]
# print("GCD of scan restarts:", np.gcd.reduce(idx[:-1]+1), 
#       "\nThis should be a multiple of the scan size if it's constant")
# rs=[]
# sn=[]
# n=0
# for s,e in zip(idx[:-1:2],idx[1::2]):
#     rays0[s+1:e+1]['idx']=0
#     rs.append(rays0[s+1:e+1])
#     sn.append(np.zeros(e-s)+n)
#     n+=1
# rays1=np.hstack(rs)
# scannum1=np.hstack(sn)
# # scannum1=scannum1[rays1['r']>=0]
# # rays1=rays1[rays1['r']>=0]

# rs=[rays0[:idx[0]+1]]
# sn=[]
# n=0
# for s,e in zip(idx[1::2],idx[2::2]):
#     rays0[s+1:e+1]['idx']=1
#     rs.append(rays0[s+1:e+1])
#     sn.append(np.zeros(e-s)+n)
#     assert(len(rays0[s+1:e+1])%1081==0)
#     n+=1

# rays2=np.hstack(rs)
# scannum2=np.hstack(sn)

# print("Timestamp of first ray:",rays1['ts'][0])

%matplotlib widget
# plt.plot(rays1['sx'],',')
# plt.plot(rays0['ts']-rays0['ts'][0],rays0['sx'],',')
# t=rays1['ts']-rays1['ts'][0]
# plt.plot(t,rays0['th']-300*t,',')
# plt.plot(rays1['ts'],np.mod(rays1['th']-2*np.pi*t/0.025025,2*np.pi),',')
# plt.plot(rays1['ts'],rays1['th'],',')
# plt.plot(rays1['ts'][540::1081],rays1['th'][540::1081],',')
# plt.plot(rays0['r']>0,',')
# _=plt.hist(np.diff(rays0['sx']),1000,range=[-.001,.001])
# plt.plot(np.arange(len(rays0))[restarts]%2162,',')
# plt.plot(np.diff(rays0[restarts]['ts'][::2]),',')

thtmp = rays1['th'].reshape(-1,1081)
rtmp = rays1['r'].reshape(-1,1081)



# print(rays2['th'].reshape(-1,1081)[:,540][0]-rays1['th'].reshape(-1,1081)[:,540][0])
c1x=rays1['ts'].reshape(-1,1081)[:,540]-rays1['ts'][0]
c1y=np.unwrap(thtmp[:,540])
c2x=rays2['ts'].reshape(-1,1081)[:,540]-rays2['ts'][0]
c2y=np.unwrap(np.mod(rays2['th'].reshape(-1,1081)[:,540]+3.145+np.pi,2*np.pi)-np.pi)
plt.plot((lp['ts']-lp['ts'][0])/1e6,lp['th']+c1y[0], 's')
plt.plot(c1x, c1y,'k.')
plt.plot(c2x, c2y,'r.')
plt.title("Plot of reference ray angle over time for both sensors")


plt.figure()
d1y=np.diff(c1y)/np.diff(c1x)
d1x=(c1x[:-1]+c1x[1:])/2
plt.plot(np.diff(d1y)/np.diff(d1x),'.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<ipython-input-12-f0d3a60601ae>:71: RuntimeWarning: invalid value encountered in true_divide
  d1y=np.diff(c1y)/np.diff(c1x)


In [13]:
%matplotlib widget
# plt.plot(rays1['ts'],rays1['sx'],',')
# plt.plot(rays2['ts'],rays2['sx'],',')

dx=np.diff(rays1['sx'])
dy=np.diff(rays1['sy'])
dd=np.sqrt(dx**2+dy**2)
dth=np.diff(rays1['th'])
dt=np.diff(rays1['ts'])
intertime=0.5*rays1['ts'][:-1]+0.5 * rays1['ts'][1:]


# plt.plot(intertime, dd/dt,',')
plt.plot( np.sort(dth/dt),',')
# plt.plot(rays1['ts'],rays1['th'],',')

# plt.plot(np.unwrap(rays1['th'])/np.pi,rays1['sx'],',')
# plt.plot(np.sort(np.diff(rays0['sx'])))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [14]:
## This cell tests for gyro scale error. 
#  If the plot shows a clear linear trend when GYRO_SCALE=1.0,
#  there is scale error. Adjust GYRO_SCALE until there is no 
#  clear linear trend. This corresponds to the amount the gyro 
#  needs to be multipled by.
#
# Input is "rays1" which is the rays from a single scanner.
# Right now it is hardcoded that a scan is 1081 rays.
# First scan is assumed to be stationary. If it's not, things get a little weird.

GYRO_SCALE=1.0 #0.95
GYRO_OFFSET = 0.0005#0.0014 # this number is 0.0005 even for perfect odometry! I wonder why?

%matplotlib widget
scans=rays1.reshape(-1,1081) #organize into scans

# Fill in the angles of the first scan to find offset angles of each ray
offth = scans['th']
tmp=scans['th'][0] # first scan angles
idxtmp=np.arange(len(tmp))[tmp!=0]
tmp2=tmp[tmp!=0] # first scan angles that are validish
    # Add in endpoints
slope=(tmp2[-1]-tmp2[0])/(idxtmp[-1]-idxtmp[0]) # Use the average slope to extrapolate endpoints
intercept=tmp2[0]-idxtmp[0]*slope
idxtmp=np.hstack((0,idxtmp,10000))
tmp2=np.hstack((intercept,tmp2,10000*slope+intercept))
offsets=np.interp(np.arange(len(tmp)),idxtmp,tmp2,left=0,right=0)

# Subtract the offsets to find the robot angle over tme
rotor=scans['th']-offsets
decent_angles=np.unwrap(rotor.ravel()[scans['r'].ravel()>0])
print(scans['ts'][0][0])
decent_ts=scans['ts'][scans['r']>0].ravel()
delt=np.diff(decent_angles,prepend=0)
slop=np.diff(decent_angles)/np.diff(decent_ts)
dupls=np.arange(len(slop))[np.diff(decent_ts)>.005] # jumps in timestamp indicating edge of scan
slop[dupls]=(slop[dupls+1]+slop[dupls-1])/2 # smooth out derivative at jumps
slop_rescale = (slop*GYRO_SCALE-GYRO_OFFSET)
incrs=slop_rescale*np.diff(decent_ts) # factor for correcting gyro scale error
regen_ang=np.cumsum(np.hstack((decent_angles[0],incrs)))

plt.figure()
plt.plot(offsets,'.')
plt.figure()
plt.plot(decent_ts,decent_angles,',')

plt.figure()
plt.plot(decent_angles,decent_angles-regen_ang)
plt.title("Adjust GYRO_SCALE to remove slope from this line")

plt.figure()
plt.plot(decent_ts,decent_angles)
plt.plot(decent_ts,regen_ang)
plt.title("Adjust GYRO_OFFSET to make flat areas in both lines flat")

plt.figure()
plt.plot(decent_ts,decent_angles-regen_ang)
# At end of cell:
# "decent_angles" is the angles from SLAM, using original odometry/IMU to do the slope within the scan
# "regen_ang" is the angles using only odometry/IMU, corrected for scale and bias
# "decent_ts" is the timestamps shared by both

1479396473.584265


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [15]:
%matplotlib widget
# plt.plot(decent_ts,decent_angles)
plt.plot(decent_ts,regen_ang,',')

its = np.linspace(decent_ts[0],decent_ts[-1], 1.29*len(scans))
print("timestep",np.diff(its)[0])
iang = np.interp(its,decent_ts,decent_angles)
plt.plot(its,iang,'.')
     
kernel_size = 10
kernel = np.ones(kernel_size) / kernel_size
smoothed = np.convolve(np.hstack((iang,iang[::-1])), kernel, mode='same')[:len(iang)]#poor man's 'mirror' end conditions
smoothed2 = np.convolve(smoothed, kernel, mode='same')
smoothed3 = np.convolve(smoothed2, kernel, mode='same')
comp_smooth=smoothed2+(smoothed2-smoothed3)*2

# help(plt.hist)
plt.plot(its,smoothed)
# plt.plot(its,smoothed2)
# plt.plot(its,smoothed3)
# plt.plot(its,comp_smooth,'.')
# plt.plot(its,iang-comp_smooth)

# plt.hist(np.diff(smoothed)*10,1000,(-0.07,0.07))
# plt.hist(iang-smoothed,1000,(-0.07,0.07))
ira=np.interp(its,decent_ts,regen_ang)
# plt.plot(its,(ira-comp_smooth)*100)
# plt.plot(its,(ira-iang)*100)
plt.plot(lp['ts']/1e6,np.unwrap(lp['th'])-np.mean(np.unwrap(lp['th'])), markersize=2)


plt.figure()
tmp=(np.diff(smoothed,prepend=0)+np.diff(smoothed,append=smoothed[-1]))*10
# plt.plot(its,(np.diff(smoothed,prepend=0)+np.diff(smoothed,append=smoothed[-1]))*10)
plt.plot(its,iang-ira,'.')
plt.plot(its,np.diff(iang,prepend=0),'.')
# plt.plot(its,smoothed-ira)

# plt.plot(its,smoothed-ira -tmp*.7)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

timestep 0.025114774703979492


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [16]:
# No idea what this cell did. Disabling.

# %matplotlib widget
# # plt.plot(decent_ts,decent_angles-regen_ang,',')

# slop_rate=.030/100
# tosum = np.hstack((decent_angles[0],incrs))
# dts = np.hstack((0,np.diff(decent_ts)))
# appr = regen_ang

# ang=0
# cors=[]
# angs=regen_ang.copy()
# for i in range(len(tosum)):
#     ang+=tosum[i]
#     corr=np.sign(decent_angles[i]-ang)*(dts[i]*slop_rate+0.1*np.abs(tosum[i]))
# #     print(decent_angles[i],ang,end=':')
#     ang+=corr
#     angs[i]=ang
# #     print(ang)
# fig, axs = plt.subplots(2,sharex=True)
# axs[0].plot(decent_ts,angs,',')
# axs[0].plot(decent_ts,decent_angles,',')
# axs[1].plot(decent_ts,np.cumsum(np.sign(decent_angles-angs)))
    
    
# # for i in range(10):

In [17]:
# No idea what this cell did. Disabling.

# %matplotlib widget
# for i in range(11):
#     corr = np.sign(decent_angles-appr)*dts*slop_rate

#     appr=np.cumsum(tosum+corr)
#     print(np.sum(np.abs(decent_angles-appr)))


# plt.plot(decent_ts,decent_angles-appr,',')
# print(np.sum(np.abs(decent_angles-appr)))
# # plt.plot(corr)


In [18]:
# No idea what this cell did. Disabling.

# %matplotlib widget

# plt.plot(decent_ts,np.hstack((0,slop_rescale)),',')

In [19]:
rays=rays0
print("orininal rays:\t",rays.shape)
r=rays['r']
keep=(r<10) & (r>0)
rays=rays[keep]
print("kept rays:\t",rays.shape)

orininal rays:	 (33653692,)
kept rays:	 (27761887,)


In [27]:
import ipywidgets
%matplotlib widget
from ipywidgets import interact, interact_manual
from IPython.display import display,clear_output
# rays2=rays1
# https://towardsdatascience.com/interactive-controls-for-jupyter-notebooks-f5c94829aee6
# reveals issues: https://stackoverflow.com/questions/61715937/how-to-enable-matplotlib-blitting-to-create-fast-realtime-graph-in-jupyterlab
# 
number = 300
plt.plot(rays['ex'][number*1081:(number+1)*1081],rays['ey'][number*1081:(number+1)*1081],'r,',c=[.5,.5,.5,.5])

@interact(number=(2,8000,1))
def display_scan(number=127):
    xlim=plt.gca().get_xlim()
    ylim=plt.gca().get_ylim()
    plt.gcf().clear()
    
#     plt.plot(rays2['ex'][number*1081:(number+1)*1081],rays2['ey'][number*1081:(number+1)*1081],'r,',c=[.5,.5,.5,.5])
    number -= 1
    plt.plot(rays2['ex'][number*1081:(number+1)*1081],rays2['ey'][number*1081:(number+1)*1081],'r.')
    number += 1
    plt.plot(rays2['ex'][number*1081:(number+1)*1081],rays2['ey'][number*1081:(number+1)*1081],'g.')
    number +=1 
    plt.plot(rays2['ex'][number*1081:(number+1)*1081],rays2['ey'][number*1081:(number+1)*1081],'b.')
    plt.plot(rays2['ex'][number*1081:(number)*1081+768],rays2['ey'][number*1081:(number)*1081+768],'c.')
    if rays2['ts'][number*1081]-rays2['ts'][(number-1)*1081]>1/30:
        plt.plot(rays2['ex'][number*1081:(number)*1081+384],rays2['ey'][number*1081:(number)*1081+384],'k.')
        plt.title('Timestamp jump = Long (1)')
    else:
        plt.plot(rays2['ex'][number*1081:(number)*1081+384],rays2['ey'][number*1081:(number)*1081+384],'k.')
        plt.title('Timestamp jump = Short (0)')
    number -=1
    plt.plot(rays2['sx'][number*1081:(number+1)*1081],rays2['sy'][number*1081:(number+1)*1081],'b.')

    if xlim[1]!=1:
        plt.xlim(xlim)
        plt.ylim(ylim)
#     plt.gcf().canvas.draw()
    s=plt.gcf().canvas
    _, _, w, h = s.figure.bbox.bounds
#     s._png_is_old = True
#     s.handle_refresh(1)
#     print(s._png_is_old)
    s.manager.resize(w, h)
    
#     plt.gcf().canvas.set_image_mode('full')
    

# plt.figure()
# for n in range(166,200):
#     plt.plot(rays2['ey'][n*1081:(n+1)*1081])
# plt.plot(rays2['ey'][198*1081:(198+1)*1081]) # weird jump at 383 to 384. Appears to repeat several data points (13?)


# DISCOVERY: The hokuyo has a glitch where sometimes the points [384:] are from one scan in the future relative to the early points!
# DISCOVERY2: Sometimes its the first points that go to the future!
# DISCOVERY3: It happens to both sensors
# DISCOVERY4: Actually there are at least 3 blocks of points [0:384][384:768][768:] that can jump to the future, as well as the timestamp. None of the 4 have a clear relation
# I wonder if the possibility of this is detectable from the timestamps? Like does it always happen before or after a missing scan? A: seems unpredictable :-(
# Also I notice that it can persist for a couple of scans, so that several in a row are split.

# Laser id 0
# Observations start at 2245
#                             a      a|
# # 5678901234567890123456789012345678|901234567890123456789
# S 0001100100110110011011010!00!00!!0|!!0000110110010011
# C 011001001000100010010?100100100010|0100110010010001001
# B 0101000100100100010010010010010001|001010100100010010 // Higher rays jumped (not sure, but mostly [384:768]
# A 1100010010001001000100100100100100|100010001001001000 // [0:384] jumped
# Y 0101001001001000100100100010010010|010011001000100100 // timestamp jumped

# Laser id 0
#  2590
#  90123456789012345678901234567890
#Y 101000010000100000100001000001
#C 101000010001000001000001000001
#B 100100001000001000010000010000
#A 100000100010000010000010000010



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

interactive(children=(IntSlider(value=127, description='number', max=8000, min=2), Output()), _dom_classes=('w…

In [15]:
# utility to find the ray nearest a certain point
def nearray(rays, x,y):
    return np.argmin((rays['ex']-x)**2+(rays['ey']-y)**2)

def plotcolored(x,y,phi):
    # plots multicolored data with phi indicating color angle
    rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T
    idx = np.round(phi%(2*np.pi)/np.pi*30) #30 divisions for now
    for i in np.unique(idx):
        plt.plot(x[idx==i],y[idx==i],',',c=rgb_cycle[idx==i][0])

In [20]:
%matplotlib widget
cut=0
raysd=rays[(rays['i']>=cut)]#&(rays['idx']==1)]
span=540
# plt.plot(raysd['ex'][5925215-span:5925215+span],raysd['ey'][5925215-span:5925215+span],'r,')
# print(raysd['ex'][11706031])
plt.gca().set_facecolor('xkcd:black')
cycle_time = 10.0
phi=(raysd['ts']-raysd['ts'][0])*np.pi*2/cycle_time
rgb_cycle = np.vstack((            # Three sinusoids
    .5*(1.+np.cos(phi          )), # scaled to [0,1]
    .5*(1.+np.cos(phi+2*np.pi/3)), # 120° phase shifted.
    .5*(1.+np.cos(phi-2*np.pi/3)))).T
plotcolored(raysd['ex'],raysd['ey'],phi)
# line=plt.plot(raysd['ex'],raysd['ey'],',')#s=1,linewidth=0,c=rgb_cycle,antialiased=False)
# help(line[0])

lpc = (lp['ts']>np.min(raysd['ts'])) & (lp['ts']<np.max(raysd['ts']))
plt.plot(lp['x'][lpc],lp['y'][lpc])
print(np.sum(lpc>0))

# # Plot Odometry
# plt.plot(rays['sx'][::10],rays['sy'][::10],'r,')
# plt.plot(odo['x'],odo['y'], ',')
# plt.plot(lp['lx'],lp['ly'], 'c.')
# plt.gca().grid('on')
plt.gca().set_aspect('equal','box')

# #Plot a slice of interest
# s1=533456
# s2=534456
# plt.plot(rays1['ex'][s1:s2],rays1['ey'][s1:s2],'g.')
# plt.plot(rays1['sx'][s1:s2],rays1['sy'][s1:s2],'b.')


# plt.plot(raysd['ex'][slc:10000000],raysd['ey'][slc:10000000],'b,')


# plt.plot(rays1['sx'],rays1['sy'],'r,')
# plt.plot(raysd['sx'][1::2],raysd['sy'][1::2],'k,')
# plt.plot(raysr['sx'][::2],raysr['sy'][::2],'r.-')
# plt.plot(raysr['sx'][1::2],raysr['sy'][1::2],'k.-')
# plt.show()
# plt.figure()
# plt.plot(raysd['th'],'b,')



ags=np.argsort(odo['ts'])

# plt.plot(odo['ts'][:-1]/1e6,1/40*np.diff(odo['x'])/(np.diff(odo['ts'])/1e6),',')
# plt.plot(odo['ts'][:-1]/1e6,1/40*np.diff(lp['th'])/(np.diff(lp['ts'])/1e6),'r,')
# plt.plot(odo['ts']/1e6,odo['th'])

print(nearray(raysd,16.08,-2.72))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

31061
23429771


In [23]:
%matplotlib widget

print(lp[0:15])
plt.plot(lp[4:50]['x'],lp[4:50]['y'],'b.')
sloc = np.mean(np.vstack((lp[15:50]['x'],lp[15:50]['y'],lp[15:50]['th'])),axis=1)
print(sloc)
plt.plot(sloc[0],sloc[1],'r.')
print(list(np.round(sloc,8)))

[(4743979136498050342, 38.492737, 68.95678 , -0.03687911, 4.5729974e-41, 38.92543 , 68.75069 )
 (4743979136498050342, 38.492737, 68.95678 , -0.03687911, 4.5729974e-41, 38.92543 , 68.75069 )
 (4743979136498076850, 38.492737, 68.95678 , -0.03687911, 4.5729974e-41, 38.112957, 69.240974)
 (4743979136498076850, 38.492737, 68.95678 , -0.03687911, 4.5729974e-41, 38.112957, 69.240974)
 (4743979136498286033, 38.544315, 68.978806, -0.03882421, 2.3509887e-38, 38.16509 , 69.26374 )
 (4743979136498365230, 38.52776 , 69.00385 , -0.04929638, 4.5729974e-41, 38.957863, 68.792404)
 (4743979136498390995, 38.557693, 69.01329 , -0.03992172, 4.5729974e-41, 38.17878 , 69.29864 )
 (4743979136498470192, 38.54246 , 69.0203  , -0.0418326 , 4.5729974e-41, 38.97413 , 68.812065)
 (4743979136498495958, 38.602333, 69.026184, -0.04493878, 4.5729974e-41, 38.224857, 69.31343 )
 (4743979136498574924, 38.554256, 69.01455 , -0.04249115, 4.5729974e-41, 38.98579 , 68.80603 )
 (4743979136498600920, 38.535244, 69.021   , -0.04

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ 3.8549557e+01  6.9022667e+01 -4.1704800e-02]
[38.549557, 69.02267, -0.0417048]


In [22]:
#Debug Odometry directly

%matplotlib widget

t=odo['ts']/1e6
x=odo['x']
y=odo['y']
th=odo['th']
ex = x - np.sin(th)*4
ey = y + np.cos(th)*4

plt.plot(ex,ey,'.')

ex = x + np.cos(th)*4
ey = y + np.sin(th)*4

plt.plot(ex,ey,'.')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [21]:
## Find the max extent of the map
np.max(rays['ex'])
for c in ['ex','ey']:
    print(c, np.min(rays[c]),np.max(rays[c]), np.max(rays[c])-np.min(rays[c]))

ex -23.340164 30.148087 53.48825
ey -11.5921545 75.36302 86.95518


In [22]:
## Figure out rasterization
# Floor of float coords is integer coords
# Images are y flipped

def homp(T,pts):
    tmppts = pts @ T[:-1,:-1].T + T[:-1,-1]
    denom = pts @ T[-1:,:-1].T + T[-1,-1]
    return tmppts/denom


CELLS_PER_M = 100.0

formatter = {'float':lambda x:np.format_float_positional(x,precision=2,fractional=True,trim='-',pad_left=8,pad_right=2)}

def get_T_px_f_with_extent(rays, debug=False):
    # The corners in world space
    f_corns = np.array(
        [[np.min(rays['ex']),np.min(rays['ey'])],
         [np.max(rays['ex']),np.max(rays['ey'])]])
    T_pxc_f = np.array(
        [[1*CELLS_PER_M, 0             , 0],
         [0            , -1*CELLS_PER_M, 0],
         [0            , 0             , 1]])
    pxc_corns=homp(T_pxc_f,f_corns)
    pxc_topleft = np.array([  np.floor(np.min(pxc_corns[:,0])), np.floor(np.min(pxc_corns[:,1]))  ]) #inclusive
    pxc_btmright= np.array([  np.floor(np.max(pxc_corns[:,0])), np.floor(np.max(pxc_corns[:,1]))  ]) + 1 #exclusive
    pxc_extent = pxc_btmright-pxc_topleft

    T_px_f = T_pxc_f.copy()
    T_px_f[:-1,-1] = -pxc_topleft

    px_corns=homp(T_px_f,f_corns)
    px_topleft = np.array([  np.floor(np.min(px_corns[:,0])), np.floor(np.min(px_corns[:,1]))  ]) #inclusive
    px_btmright= np.array([  np.floor(np.max(px_corns[:,0])), np.floor(np.max(px_corns[:,1]))  ]) + 1 #exclusive
    px_extent = px_btmright-px_topleft

    assert (np.all(pxc_extent == px_extent))

    if debug:
        print("fcorns\n",f_corns)

        for var in ['pxc_corns',
                    'pxc_topleft',
                    'pxc_btmright',
                    'pxc_extent',
                    'px_corns',
                    'px_topleft',
                    'px_btmright',
                    'px_extent',
                    ]:
            print(var+'\n', eval(var))
    return T_px_f, px_extent.astype('int64')

with np.printoptions(formatter=formatter):
    T_px_f, px_extent = get_T_px_f_with_extent(rays, debug=True)

fcorns
 [[     -23.34      -11.59]
 [      30.15       75.36]]
pxc_corns
 [[   -2334.02     1159.22]
 [    3014.81    -7536.3 ]]
pxc_topleft
 [   -2335       -7537   ]
pxc_btmright
 [    3015        1160   ]
pxc_extent
 [    5350        8697   ]
px_corns
 [[       0.98     8696.22]
 [    5349.81        0.7 ]]
px_topleft
 [       0           0   ]
px_btmright
 [    5350        8697   ]
px_extent
 [    5350        8697   ]


In [23]:
# rasterize hits

def get_raster_coords(rays, T_px_f):
    # implicit inputs: T_px_f
    f_ept = np.vstack((rays['ex'].ravel(),rays['ey'].ravel())).T.copy()
    px_coords = homp(T_px_f, f_ept)
    px_coords = px_coords.reshape(rays['ex'].shape+(2,))
    return px_coords

px_coords = get_raster_coords(rays0, T_px_f)

bad0=rays0['r']<=0
bad0|=rays0['r']>59

img = np.zeros(px_extent[::-1])

# clip out bad pixels
bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
px_coords = px_coords[bad==0,:]




np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
%matplotlib widget

imsc=img.copy()
imsc[img!=0]=np.log10(img[img!=0])+0.301



plt.imshow(imsc)
import cv2
cv2.imwrite("rendered_image_linear.png",img.astype(np.uint16))
print('maxhits=',np.max(img))
plt.figure()
plt.plot(img[900,:],'.')
# plt.hist(rays0['r'],100)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

maxhits= 1229.0


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# It turns out that gaussian blurring and selecting where >1 produces a very good approximation of where the walls are
# without much reflection. This cell does that calculation
%matplotlib widget

def produce_wall_mask(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]

    np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
    
    gaussian_func = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)

    local_mean = gaussian_func(img)
    return local_mean>1.0
wall_mask=produce_wall_mask(rays0)
plt.imshow(wall_mask)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [24]:
%matplotlib widget

def produce_fwhm_mask(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]

    np.add.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 1)
    
    kernel = np.ones((10,10), np.uint8)
    dil=cv2.dilate(img, kernel, borderType=cv2.BORDER_DEFAULT)
    plt.imshow((img>=dil/2)*(img>3))

    return (img>dil/2)
fwhm_mask=produce_fwhm_mask(rays0)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [87]:
# load gt data and estimate normals
%matplotlib widget
import cv2

gtrgb=cv2.imread("gtmap.png")
# plt.imshow((gtrgb[:,:,1]>0)+0.5*fwhm_mask)
gtwall = gtrgb[:,:,1]>0
gtwall_coords = np.array(np.nonzero(gtwall))[::-1].T+0.5


# see the distribution of coordinates by distance
from scipy.spatial import KDTree
tree= KDTree(gtwall_coords)
d,nn=tree.query(px_coords[::360],distance_upper_bound=10.0)
# d,nn=tree.query(px_coords[:108100],distance_upper_bound=10.0)

# histogram out to +- 0.5 cell spacing is linear ramp, because it is an expanding circle. Beyond that it should approach gaussian in absense of localization error
# arclen=arcsin(CELL_SPACING/2/x)*4/(2*pi)*(2*pi*r)
# density = exp(-2*(x/SD)**2) # where x points in the normal direction
REGRESSION_SD=2.0
plt.hist(d[d<10],100)
x=np.linspace(0.01,10,1000)
plt.plot(x,np.arcsin(np.clip(0.5/x,0,1))*4*x*np.exp(-(x/REGRESSION_SD)**2)*1000)#1000 scale factor for visualization, adjust as needed

# plt.plot(px_coords[:108100,0],px_coords[:108100,1],',')
plt.figure()
plt.plot(gtwall_coords[:,0],gtwall_coords[:,1],',')
# x=np.linspace(0,10)
# plt.plot(x,np.exp(-(x/4)**2)*10000)
# print(nn)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [20]:
%matplotlib widget

def produce_max_inten_image(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    bad0|=np.abs(rays0['th']+1.627-np.pi*3/2)>.0305
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]
    
    inten = rays0['i'][bad==0]

    np.maximum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), inten)
#     plt.imshow(img)
    f2=np.all(np.abs(np.floor(px_coords)-[2101,4168])<1000,axis=1) #top right:[3773,960] , mid left:[363,3871], alcove:[2018,2032], inner pillar:[2483,713]
    print(np.sum(f2))
    
    angs = np.interp(rays0['ts'],decent_ts,decent_angles)
#     plt.plot(rays0['ts'][bad==0][f2],(rays0['th'])[bad==0][f2],'.')
#     plt.plot(rays0['r'][bad==0][f2],rays0['i'][bad==0][f2],'.')
    plt.plot(rays0['th'][bad==0][f2],rays0['i'][bad==0][f2]+rays0['r'][bad==0][f2]*0,',')
#     plt.plot(rays0['r'][bad==0][f2],rays0['i'][bad==0][f2]+rays0['r'][bad==0][f2]*0,',')

#     plt.plot(rays0['ts'][bad==0][f2],rays0['i'][bad==0][f2],'.')
#     plt.plot(rays0['ts'][bad==0][f2],rays0['r'][bad==0][f2]*100,'.')
#     plt.plot(rays0['ts'][bad==0][f2],rays0['th'][bad==0][f2]*1000,'.')


#     plt.plot(rays0['i'][bad==0][f2],'.')

    plt.plot([2.37,7.93],[3105,1891])
    print(np.diff([3105,1891])/np.diff([2.37,7.93]))
    
    return img
maxinten=produce_max_inten_image(rays0)
plt.figure()
plt.imshow(maxinten>0)

14929


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[-218.34532374]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [37]:
#try some local enhancement:
%matplotlib widget

import cv2
gaussian_func = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)
DECENT_LINE_FRACTION = 0.7

local_mean = gaussian_func(img)

line_img = np.zeros((300,300))
line_img[150]=100
print("expected best for line:",100/np.max(gaussian_func(line_img)))
# plt.imshow(np.clip(img/(local_mean+1),0,10)>.8)
kernel_max = 1/np.max(img/(local_mean+0.0000001)) # appoximation of the center value of the gaussian kernel
gaussian_without_center=(local_mean-img*kernel_max)/(1-kernel_max)#mean estimate ignoring center pixel
denom=np.clip(local_mean,.0001,1e9) # considered gaussian_without_center, but nope 
qual_analog=np.clip(np.clip(img-np.sqrt(img),0,1e9)/denom,0,50)#lower bound of conf interval scales with sqrt
determiner = (qual_analog>0.7)*(img>0)
# plt.imshow((qual_analog>0.7) +(img>0))  
plt.imshow(img>0)
plt.figure()
plt.plot(img[630:645,1257],'.')
print(np.unravel_index(np.argmax(img),img.shape))
# plt.imshow(qual_analog>.7)
# plt.plot(np.sort(qual_analog.ravel()))
print("best observed:",np.max(img)/local_mean.ravel()[np.argmax(img)])
# plt.imshow((img>2)*0.5+(np.clip(img/(local_mean+1),0,10)>.8)*0.5)

expected best for line: 7.519884823893004


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(858, 1257)
best observed: 3.6316730282930405


In [23]:
# find local maxima for realigning scans
%matplotlib widget

kernels=np.array([
        [
            [0,0,1],
            [0,1,0],
            [1,0,0.0],
        ],
        [
            [0,1,0],
            [0,1,0],
            [0,1,0],
        ],
        [
            [1,0,0],
            [0,1,0],
            [0,0,1],
        ],
        [
            [0,0,0],
            [1,1,1],
            [0,0,0],
        ],
    ]).astype(np.uint8)
sol=np.zeros_like(local_mean)
gaussian_func_s = lambda img:cv2.GaussianBlur(img,(61,61),sigmaX=3,borderType=cv2.BORDER_DEFAULT)
ridgeimg=gaussian_func_s(img)
for kernel in kernels:
    sol+=(ridgeimg==cv2.dilate(ridgeimg, kernel, iterations=1))
plt.imshow(np.clip(sol,0,8)*(local_mean>0.5)+(img>20))
print("nonmax",np.sum(img>0),np.sum((img>0)*(sol>0)),np.sum(img>1),np.sum((img>1)*(sol>0)))
print(np.sum((sol>0)*(local_mean>0.5)))

cores=np.array(np.nonzero((sol>0)*(local_mean>0.5))).T+[0.5,0.5]

from scipy.spatial import KDTree

tree=KDTree(cores)
T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates

In [31]:
print(len(px_coords))

d,matchidx = tree.query(px_coords[:,::-1][:100000], eps=2,distance_upper_bound=20) # can only query 10k pts per second :-(
error = px_coords[:,::-1]-cores[matchidx]
plt.figure()
plt.hist(d[rays0['r']>0], )


18135937


IndexError: index 126014 is out of bounds for axis 0 with size 126014

In [22]:
def cimshow(img):
    dpi = 200 # Affects only tick and text sizes
    margin = 0.05/(np.max(img.shape)/100) # (5% of the width/height of the figure...)
    xpixels, ypixels = img.shape[1], img.shape[0]

    # Make a figure big enough to accomodate an axis of xpixels by ypixels
    # as well as the ticklabels, etc...
    figsize = (1 + margin) * ypixels / dpi, (1 + margin) * xpixels / dpi

    fig = plt.figure(figsize=figsize, dpi=dpi)
    # Make the axis the right size...
    ax = fig.add_axes([margin, margin, 1 - 2*margin, 1 - 2*margin])

    ax.imshow(img, interpolation='none')
    return ax

In [93]:
%matplotlib widget
import scipy.signal as signal

inten=rays1['i'].reshape(-1,1081)
plt.imshow(inten[:3000])

#(img>2)*0.5+(np.clip(img/(local_mean+1),0,10)>.8)*0.5
px_xy = get_raster_coords(rays2.reshape(-1,1081), T_px_f)
bad=rays2['r'].reshape(-1,1081)<=0
bad|=rays2['r'].reshape(-1,1081)>59
print(np.sum(bad))
bad|=px_xy[:,:,0]<0
bad|=px_xy[:,:,1]<0
# bad|=np.any(px_xy,axis=2)<0
print(np.sum(bad))
bad|=px_xy[:,:,0]>img.shape[1]
print(np.sum(bad))
bad|=px_xy[:,:,1]>img.shape[0]
print(np.sum(bad))


px_inds = tuple(np.rollaxis(np.floor(px_xy)[...,::-1].astype('int32'),-1))
# print(px_inds.shape)
flatinds = np.ravel_multi_index(px_inds,img.shape,mode='clip') # need to hand linearize to specify what to do with bad inds
img[0,:]=0
img[:,0]=0
img_samp = img.ravel()[flatinds].reshape(-1,1081)
mean_samp = local_mean.ravel()[flatinds].reshape(-1,1081)
detr_samp = determiner.ravel()[flatinds].reshape(-1,1081)

qual=np.clip(np.clip(img_samp-np.sqrt(img_samp),0,1e9)/(mean_samp+0.00001),0,10)
metric = np.sum(qual>1.0,axis=1)/np.sum(bad==False,axis=1)

plt.figure()
plt.plot(metric)
# plt.plot(np.sum(img_samp>10,axis=1))
# plt.plot(metric,'.')

cutoff=signal.order_filter(metric,np.ones(61),45)
metric_flattened = metric-cutoff
metric_flattened[:31]=metric_flattened[31]
metric_flattened[-31:]=metric_flattened[-31]
to_keep=metric_flattened
to_keep=to_keep.reshape(-1,1)+np.zeros(1081)
# print(to_keep[100:120,100:110])
plt.plot(cutoff)
# plt.plot(metric_flattened)
plt.plot(np.sort(metric))

# display filtered points
imf1=np.zeros_like(img)
# px_inds=np.array(px_inds)
# np.maximum.at(imf.ravel(), flatinds, (metric-signal.order_filter(metric,np.ones(21),5)).ravel())
# np.maximum.at(imf1.ravel(), flatinds[bad==False], -to_keep[bad==False])
np.maximum.at(imf1.ravel(), flatinds[bad==False], to_keep[bad==False]+1)

# np.maximum.at(imf1.ravel(), flatinds, to_keep==51)

# np.add.at(imf1.ravel(), flatinds, (-1.0*(to_keep<0)))
# np.add.at(imf1.ravel(), flatinds[bad==False], (1.0*(to_keep>0))[bad==False])
# np.add.at(imf1.ravel(), flatinds, 1.0)

im=np.clip(imf1,-100,1)+determiner*0.2
plt.figure()
plt.imshow(img/(local_mean+10))
ce=5112
plt.plot(px_xy[ce-1,:,0],px_xy[ce-1,:,1],'r.')
plt.plot(px_xy[ce,:,0],px_xy[ce,:,1],'.')
plt.plot(px_xy[ce+1,:,0],px_xy[ce+1,:,1],'.')

# cimshow(im[:2000,:5000])
print(metric_flattened[2198])
print(np.max(img/(local_mean+10)))
goodgrad=img/(local_mean+10)
cv2.imwrite('tmp.png',goodgrad/3.8*256)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

1676189
1692366
1698652
1698657


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

-0.10840484271694317
3.861872264348472


True

In [92]:
# rasterize hits by time

%matplotlib widget
colorang=rays0['ts']
colorvec=2*np.vstack([np.cos(colorang),np.sin(colorang)])
rv=[1,0.0]
gv=[np.cos(2/3*np.pi),np.sin(2/3*np.pi)]
bv=[np.cos(4/3*np.pi),np.sin(4/3*np.pi)]
th=np.linspace(0,7)
r=np.clip(np.dot(rv,colorvec),-1,1)*0.5+0.5
g=np.clip(np.dot(gv,colorvec),-1,1)*0.5+0.5
b=np.clip(np.dot(bv,colorvec),-1,1)*0.5+0.5
# plt.plot(r)
# plt.plot(g)
# plt.plot(b)


f_ept = np.vstack((rays0['ex'],rays0['ey'])).T.copy()
bad0=rays0['r']<=0
bad0|=rays0['r']>5

im3=np.zeros(list(px_extent[::-1])+[3])
px_coords = homp(T_px_f, f_ept)

px_xyrgb = np.hstack((px_coords,r[:,np.newaxis],g[:,np.newaxis],b[:,np.newaxis]))
bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
del px_coords

px_xyrgb = px_xyrgb[bad==0]
px_xy=np.floor(px_xyrgb[:,0:2]).astype('int64')

lindex = np.ravel_multi_index((px_xy[:,1],px_xy[:,0]), px_extent[::-1],mode='raise')

im3.ravel()[3*lindex]=px_xyrgb[:,2]
im3.ravel()[3*lindex+1]=px_xyrgb[:,3]
im3.ravel()[3*lindex+2]=px_xyrgb[:,4]

# plt.imshow(im3)
import cv2
cv2.imwrite('color_by_time.png',
            (np.concatenate(
                (im3[:,:,::-1]*255.9,255*np.ones(list(px_extent[::-1])+[1])),axis=2
            )*255).astype('uint16'))

True

In [ ]:
# dump every second of data into a file
timedir='time_imgs'
start_time = np.min(rays0['ts'])
duration = np.ceil(np.max(rays0['ts'])-np.min(rays0['ts']))
import cv2

def produce_basic_coords(rays0):
    import cv2
    T_px_f, px_extent = get_T_px_f_with_extent(rays) # extents and transform
    px_coords = get_raster_coords(rays0, T_px_f) # pixel coordinates
    img = np.zeros(px_extent[::-1]) #image to fill

    # clip out bad rays
    bad0=rays0['r']<=0
    bad0|=rays0['r']>59
    # clip out bad pixels
    bad = np.any(px_coords<0,axis=1) + np.any(px_coords >= px_extent,axis=1) + bad0
    px_coords = px_coords[bad==0,:]
    return px_coords

img = np.zeros(px_extent[::-1],np.uint8) #image to fill
    
   
for i in range(int(duration)):
    name = '{}.png'.format(i)
    select=rays0['ts']>=start_time+i
    select&= rays0['ts']<start_time+i+1
    px_coords = produce_basic_coords(rays0[select])
    np.maximum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 255)
    print(name)
    cv2.imwrite(name, img)
    np.minimum.at(img, tuple(np.floor(px_coords.T)[::-1].astype('int32')), 0)

    

In [ ]:
# BELOW HERE IS ALL EXPERIMENTAL STUFF
# TODO Cell: Display only the points that have a closer start point than any other start point 
# i.e. the point which is when the robot was closest to that spot.


%matplotlib widget
#as a simple approximation, lets see if using a certain offset foward and backward in time is okay
offset = 1000

def dist(pt,pts):
    return np.sqrt(np.sum((pt-pts)**2,axis=1))

sx = rays1['sx']#use rays only from one laser
sy = rays1['sy']
spt= np.vstack((sx,sy)).T

ex = rays1['ex']
ey = rays1['ey']
ept=np.vstack((ex,ey)).T

#remove far rays
rays11=rays1[dist(ept,spt)<10]
scannum11=scannum1[dist(ept,spt)<10]



#reunpack the ray that remain

sx = rays11['sx']#use rays only from one laser
sy = rays11['sy']
spt= np.vstack((sx,sy)).T

ex = rays11['ex']
ey = rays11['ey']
ept=np.vstack((ex,ey)).T

# # check that we are closer that
# fsx = sx[offset:]
# fsy = sy[offset:]
# bsx = sx[:-offset]
# bsy = sy[:-offset]
# cex = ex[offset//2:-offset//2]
# cey = ex[offset//2:-offset//2]


rsx=raysr['sx']
rsy= raysr['sy']
rpt= np.vstack((rsx,rsy)).T


probes = np.arange(0,len(sx),1000) # some points where we can get the true answer
# probes = [5025083]

def dist(pt,pts):
    return np.sqrt(np.sum((pt-pts)**2,axis=1))
sols=[] #the ones with a good closest approach
bsols=[] # the ones that never seem to have a good closest approach
stats=[]
for p in probes:
    pex=ex[p]
    pey=ey[p]
    ppt=np.array([pex,pey])
    d=dist(ppt, rpt)
    d=dist(ppt, spt)
    timecost=np.abs(scannum11[p]-scannum11)/300
#     plt.clf()
#     plt.plot(timecost)
#     plt.plot(d)
#     plt.plot(d+timecost)
    d3=dist(ppt,ept)#distance to original endpoint

#     break
    closest = np.argmin(d+timecost)
#     plt.plot(d)

#     print(closest,pex,pey)
    cpt=spt[closest]
    d2=dist(cpt,spt)#distance to best center
#     plt.plot(d2)
#     d3=dist(ppt,ept)#distance to original endpoint
#     plt.plot(d3+d2)
    cost=d3+d2
    remin=np.argmin(cost)
    print(p,remin,cost[remin])
    if cost[remin]<0.1:
        sols.append(remin)
    else:
#         plt.plot(d)
#         plt.plot(d3)
        bsols.append(remin)
#         break
    stats.append(cost[remin])
plt.plot(ex[sols],ey[sols],"b.")
plt.plot(ex[bsols],ey[bsols],"r.")

# rsx[closest],rsy[closest]

In [ ]:

%matplotlib widget
plt.close('all')

# plt.plot(rays11['ex'],rays11['ey'],'b,')

# tpti = 1000000
# tpt=np.array([rays11['ex'][tpti],rays11['ey'][tpti]])
# stpt=np.array([rays11['sx'][tpti],rays11['sy'][tpti]])
# # plt.plot(raysr['sx'][::2],raysr['sy'][::2],'r.-')
# plt.plot(raysr['sx'][1::2],raysr['sy'][1::2],'k.-')

# plt.plot(rays11['ex'][tpti],rays11['ey'][tpti],'r.')
# plt.plot(rays11['sx'][tpti],rays11['sy'][tpti],'g.')

# print(len(raysr),np.max(scannum11))
# dr=dist(stpt, rpt)
# iraysr = np.argmin(dr)
# print(iraysr,scannum11[tpti])
# plt.figure()
# plt.plot(dr)

# plt.figure()
dr=dist(spt,ept)
plt.plot(dr,'-')

def advance(line, marker, mask):
    left=line[:-1]
    line[1:]=np.minimum(left,line[1:])
    line[1:]=np.maximum(line[1:],mask[1:])
#     left[left<line[1:]]left[left<line[1:]]
    return line
# advance(dr,dr,dr-0.1)
line=dr.copy()
marker=dr
mask=dr-0.05
for i in range(100):
    line=advance(line, marker, mask)
line=line[::-1]
marker = marker[::-1]
mask = mask[::-1]
for i in range(100):
    line=advance(line, marker, mask)
line=line[::-1]
marker = marker[::-1]
mask = mask[::-1]
plt.plot(line,'-')
localmins = (line==dr)
plt.figure()
plt.plot(rays11['ex'],rays11['ey'],'b,')
plt.plot(rays11['ex'][localmins],rays11['ey'][localmins],'r.')




In [ ]:
%matplotlib widget

plt.plot(ex[sols],ey[sols],"b.")
plt.plot(ex[bsols],ey[bsols],"r.")
plt.plot(ex[probes],ey[probes],"g,")

In [ ]:
%matplotlib widget
plt.hist(stats,500)